## Laptime2Rank-evaluate-fulltest-disturbance

based on: Laptime2Rank-evaluate-fulltest

rank prediction by laptime forecasting models

support:
+ train/test split by ratio or event
+ incremental training evaluation(adjust ratio)
+ go beyond curtrack and zerotrack by modeling the track status
+ halfwin mode(0:no, 1:halfwin, 2:continous)
+ split by stage, support all events (todo)

+ disturbance analysis by adding disturbance to oracle trackstatus and lapstatus


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
import os
random.seed()
os.getcwd()
#GPUID = 1

'/scratch_hdd/hpda/indycar/notebook/13.FullTest'

In [3]:
def load_data(event, year):
    inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    #print('count of completed cars:', completed_car_count)
    #print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata

# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    #uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
    #                  "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
    #                  "last_pitted_lap","start_position","laps_led"], axis=1)
    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", 
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
        
    carnumber = set(uni_ds['car_number'])
    #print('cars:', carnumber)
    #print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    #df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    df = uni_ds[['car_number','completed_laps','rank','elapsed_time',
                 'rank_diff','time_diff',"current_status", "track_status", "lap_status"]]
    
    return df

In [100]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6

# oracle mode
MODE_ORACLE = 0  # oracle = track + lap
MODE_ORACLE_TRACKONLY = 1
MODE_ORACLE_LAPONLY = 2   
   

# oracle mode for training
MODE_NOLAP = 1   
MODE_NOTRACK = 2

# predicting mode
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8

MODE_PREDTRACK = 16
MODE_PREDPIT = 32

# disturbe analysis
MODE_DISTURB_CLEARTRACK = 64
MODE_DISTURB_ADJUSTTRACK = 128
MODE_DISTURB_ADJUSTPIT = 256


_mode_map = {MODE_ORACLE:'MODE_ORACLE',MODE_ORACLE_TRACKONLY:'MODE_ORACLE_TRACKONLY',
            MODE_ORACLE_LAPONLY:'MODE_ORACLE_LAPONLY',
             MODE_TESTZERO:'MODE_TESTZERO',MODE_TESTCURTRACK:'MODE_TESTCURTRACK',
             MODE_PREDTRACK:'MODE_PREDTRACK',MODE_PREDPIT:'MODE_PREDPIT',
            MODE_DISTURB_CLEARTRACK:'MODE_DISTURB_CLEARTRACK',MODE_DISTURB_ADJUSTTRACK:'MODE_DISTURB_ADJUSTTRACK',
            MODE_DISTURB_ADJUSTPIT:'MODE_DISTURB_ADJUSTPIT'}

def get_modestr(a):
    modestr = ''
    for key in _mode_map:
        if test_flag(a, key):
            modestr += '%s,'%(_mode_map[key])
            
    return modestr

# endpos -> vector of prediction_length
_track_pred  = {}
_track_true  = {}
def init_track_model():
    global _track_pred,_track_true
    _track_pred = {}
    _track_true  = {}
    
def get_track_model(track_rec, endpos, prediction_length, context_len=10):
    """
    return the predicted track status
    """
    global _track_pred,_track_true
    # this is the perfect track model for Indy500 2018
    track_model = [6,4,4,5,6,6,4]
    if endpos in _track_pred:
        return _track_pred[endpos]
    else:
        #get yflag lap count from the start pred point
        yflaplen = 0
        for i in range(1, context_len):
            if track_rec[- prediction_length - i] == 1:
                yflaplen += 1
            else:
                break
                
        #laps remain, fill into the future
        trackpred = np.array([0 for x in range(prediction_length)])
        
        yflap_pred = random.choice(track_model)
        if yflaplen > 0 and yflap_pred > yflaplen:
            trackpred[:(yflap_pred - yflaplen)] = 1
        _track_pred[endpos] = trackpred
        
        _track_true[endpos]  = track_rec[- prediction_length:].copy()
        
        return trackpred

    
# endpos -> vector of prediction_length
_track_adjust  = {}
def init_adjust_track_model():
    global _track_adjust
    _track_adjust = {}
    
def adjust_track_model(track_rec, endpos, prediction_length, tailpos):
    """
    input:
        tailpos ; <0 end pos of 1
    return the predicted track status
    """
    global _track_adjust
    # this is the perfect track model for Indy500 2018
    track_model = [-1,0,1]
    if endpos in _track_adjust:
        return _track_adjust[endpos]
    else:
        yflap_adjust = random.choice(track_model)
        
        #laps remain, fill into the future
        trackadjust = track_rec[-prediction_length:].copy()
        if yflap_adjust == -1:
            trackadjust[tailpos] = 0
        elif yflap_adjust == 1:
            trackadjust[tailpos] = 0
            if (tailpos + 1) <= -1:
                trackadjust[tailpos+1] = 1
        
        _track_adjust[endpos] = trackadjust
        
        return trackadjust
    
def adjust_pit_model(lap_rec, endpos, prediction_length):
    """
    input:
        tailpos ; <0 end pos of 1
    return the predicted lap status
    """
    adjust_model = [-1,0,1]
    lap_adjust = random.choice(adjust_model)
        
    #laps remain, fill into the future
    lapadjust = lap_rec[-prediction_length:].copy()
    for pos in range(0, prediction_length):
        if lapadjust[pos] == 1:
            # adjust this pit lap position
            pos_adjust = random.choice(adjust_model)

            if pos_adjust == -1:
                if (pos - 1 >= 0):
                    lapadjust[pos] = 0
                    lapadjust[pos - 1] = 1
            elif pos_adjust == 1:
                if (pos + 1 < prediction_length):
                    lapadjust[pos] = 0
                    lapadjust[pos + 1] = 1

    return lapadjust
    
# pit model is separate for each car
def get_pit_model(cuation_laps_instint, laps_instint, prediction_length):
    """
    return the predicted pit status
    """
    # this is the perfect empirical pit model for Indy500 2018
    pit_model_all = [[33, 32, 35, 32, 35, 34, 35, 34, 37, 32, 37, 30, 33, 36, 35, 33, 36, 30, 31, 33, 36, 37, 35, 34, 34, 33, 37, 35, 39, 32, 36, 35, 34, 32, 36, 32, 31, 36, 33, 33, 35, 37, 40, 32, 32, 34, 35, 36, 33, 37, 35, 37, 34, 35, 39, 32, 31, 37, 32, 35, 36, 39, 35, 36, 34, 35, 33, 33, 34, 32, 33, 34],
                [45, 44, 46, 44, 43, 46, 45, 43, 41, 48, 46, 43, 47, 45, 49, 44, 48, 42, 44, 46, 45, 45, 43, 44, 44, 43, 46]]
    pit_model_top8 = [[33, 32, 35, 33, 36, 33, 36, 33, 37, 35, 36, 33, 37, 34],
                 [46, 45, 43, 48, 46, 45, 45, 43]]
    
    pit_model = pit_model_all
    
    if cuation_laps_instint>10:
        #use low model
        pred_pit_laps = random.choice(pit_model[0])
    else:
        pred_pit_laps = random.choice(pit_model[1])
                
    #laps remain, fill into the future
    pitpred = np.array([0 for x in range(prediction_length)])
    
    if (pred_pit_laps > laps_instint) and (pred_pit_laps <= laps_instint + prediction_length):
        pitpred[pred_pit_laps - laps_instint - 1] = 1
         
    return pitpred    
    
def make_dataset_byevent(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       test_event = 'Indy500',
                       test_cars = [],  
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       half_moving_win = 0,
                       train_ratio=0.8,
                       verbose = False
                ):
    """
    split the ts to train and test part by the ratio
    
    input:
        oracle_mode: false to simulate prediction in real by 
                set the covariates of track and lap status as nan in the testset
        half_moving_win  ; extend to 0:-1 ,1:-1/2plen, 2:-plen
    
    """    
    init_track_model()
    init_adjust_track_model()
    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        if events[_data[0]] == test_event:
            test_mode = True
        
        else:
            test_mode = False
            
            
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        max_len = int(np.max(ts_len))
        train_len = int(np.max(ts_len) * train_ratio)
        
        if verbose:
            print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                if verbose:
                    print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK) or test_flag(oracle_mode, MODE_ORACLE_LAPONLY):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP) or test_flag(oracle_mode, MODE_ORACLE_TRACKONLY):                
                rec[COL_LAPSTATUS, :] = 0

            test_rec_cnt = 0
            if not test_mode:
                
                # all go to train set
                _train.append({'target': rec[run_ts,:].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [rec[COL_TRACKSTATUS,:],
                                       rec[COL_LAPSTATUS,:]]
                              }
                              )
            else:
                # reset train_len
                #context_len = prediction_length*2
                #if context_len < 10:
                #    context_len = 10
                
                context_len = train_len
                
                # multiple test ts(rolling window as half of the prediction_length)

                #step = -int(prediction_length/2) if half_moving_win else -prediction_length
                if half_moving_win == 1:
                    step = -int(prediction_length/2)
                elif half_moving_win == 2:
                    step = -prediction_length
                else:
                    step = -1
                
                #bug fix, fixed the split point for all cars/ts
                for endpos in range(max_len, context_len+prediction_length, 
                                    step):

                    #check if enough for this ts
                    if endpos > totallen:
                        continue
                    
                    track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                    lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                    
                    caution_laps_instint = int(rec[COL_CAUTION_LAPS_INSTINT, endpos -prediction_length - 1])
                    laps_instint = int(rec[COL_LAPS_INSTINT, endpos -prediction_length - 1])
                    

                    # test mode
                    if test_flag(oracle_mode, MODE_TESTCURTRACK):
                        # since nan does not work, use cur-val instead
                        track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                        #track_rec[-prediction_length:] = random.randint(0,1)
                        #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                        lap_rec[-prediction_length:] = 0
                    elif test_flag(oracle_mode, MODE_TESTZERO):
                        #set prediction part as nan
                        #track_rec[-prediction_length:] = np.nan
                        #lap_rec[-prediction_length:] = np.nan
                        track_rec[-prediction_length:] = 0
                        lap_rec[-prediction_length:] = 0        

                    # predicting with status model
                    if test_flag(oracle_mode, MODE_PREDTRACK):
                        predrec = get_track_model(track_rec, endpos, prediction_length)
                        track_rec[-prediction_length:] = predrec
                        #lap_rec[-prediction_length:] = 0
                        
                    if test_flag(oracle_mode, MODE_PREDPIT):
                        #predrec = get_track_model(track_rec, endpos, prediction_length)
                        #track_rec[-prediction_length:] = predrec
                        lap_rec[-prediction_length:] = get_pit_model(caution_laps_instint,
                                                                    laps_instint,prediction_length)
                        
                        
                    # disturbe analysis
                    if test_flag(oracle_mode, MODE_DISTURB_CLEARTRACK):
                        # clear the oracle track status
                        # future 1s in trackstatus
                        # pattern like 0 1 xx
                        for _pos in range(-prediction_length + 1, -1):
                            if track_rec[_pos - 1] == 0:
                                track_rec[_pos] = 0
                                
                    if test_flag(oracle_mode, MODE_DISTURB_ADJUSTTRACK):
                        # adjust the end position of track, or caution lap length
                        # find the end of caution laps
                        _tail = 0
                        for _pos in range(-1,-prediction_length + 1,-1):
                            if track_rec[_pos] == 1:
                                #find the tail
                                _tail = _pos
                                break
                        if _tail != 0:
                            #found
                            adjustrec = adjust_track_model(track_rec, endpos, prediction_length, _tail)
                            track_rec[-prediction_length:] = adjustrec
                        
                    if test_flag(oracle_mode, MODE_DISTURB_ADJUSTPIT):
                        # adjust the position of pit
                        if np.sum(lap_rec[-prediction_length:]) > 0:
                            adjustrec = adjust_pit_model(lap_rec, endpos, prediction_length)
                            lap_rec[-prediction_length:] = adjustrec
                        

                    _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [track_rec,lap_rec]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                                #       rec[COL_LAPSTATUS,:endpos]] 
                                 }
                              )   
                    test_rec_cnt += 1
            
            #add one ts
            if verbose:
                print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)        

### test for Indy500

In [5]:
def predict(test_ds,predictor):
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=100,  # number of sample paths we want for evaluation
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)
    print(f'tss len={len(tss)}, forecasts len={len(forecasts)}')
    
    return tss, forecasts

   
def run_prediction_ex(test_ds, prediction_length, model_name,trainid):
    with mx.Context(mx.gpu(7)):    
        pred_ret = []

        rootdir = f'../models/remote/laptime-{trainid}/'
        # deepAR-Oracle
        if model_name == 'curtrack':
            model=f'deepAR-Oracle-laptime-curtrack-indy-f1min-t{prediction_length}-e1000-r1_curtrack_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        elif model_name == 'zerotrack':
            model=f'deepAR-Oracle-laptime-nolap-zerotrack-indy-f1min-t{prediction_length}-e1000-r1_zerotrack_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]
            
        # deepAR-Oracle
        elif model_name == 'oracle':
            model=f'deepAR-Oracle-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_oracle_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # deepAR
        elif model_name == 'deepAR':
            model=f'deepAR-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_deepar_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # naive
        elif model_name == 'naive':
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  NaivePredictor(freq= freq, prediction_length = prediction_length)
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # arima
        elif model_name == 'arima':
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  RForecastPredictor(method_name='arima',freq= freq, 
                                            prediction_length = prediction_length,trunc_length=60)
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]
        else:
            print(f'error: model {model_name} not support yet!')

        return pred_ret     

In [70]:
#calc rank
def eval_rank_bytimediff(test_ds,tss,forecasts,prediction_length):
    """
    timediff models
    
    works for one event only
    
    """

    carlist = []

    # carno-lap# -> elapsed_time[] array
    forecasts_et = dict()

    ds_iter =  iter(test_ds)
    for idx in range(len(test_ds)):
        test_rec = next(ds_iter)
        #global carid
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        #print('car no:', carno)

        if carno not in carlist:
            carlist.append(carno)

        # calc elapsed time
        prediction_len = forecasts[idx].samples.shape[1]
        if prediction_length != prediction_len:
            print('error: prediction_len does not match, {prediction_length}:{prediction_len}')
            return []
        
        #forecast_laptime_mean = np.mean(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        forecast_laptime_mean = np.median(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        
        timediff_array = tss[idx].values.copy()

        #save the prediction
        completed_laps = len(tss[idx]) - prediction_len + 1
        #print('car no:', carno, 'completed_laps:', completed_laps)
        #key = '%s-%s'%(carno, completed_laps)
        #forecasts_et[key] = elapsed_time[-prediction_len:].copy()
        if completed_laps not in forecasts_et:
            forecasts_et[completed_laps] = {}
        forecasts_et[completed_laps][carno] = [timediff_array[-prediction_len:].copy(),
                                                   forecast_laptime_mean.copy()]


    # calc rank
    rank_ret = []
    for lap in forecasts_et.keys():
        #get car list for this lap
        carlist = list(forecasts_et[lap].keys())
        #print('carlist:', carlist)

        caridmap={key:idx for idx, key in enumerate(carlist)}

        #fill in data
        time_diff = np.zeros((2, len(carlist), prediction_len))
        for carno in carlist:
            carid = caridmap[carno]
            time_diff[0, carid, :] = forecasts_et[lap][carno][0].reshape((prediction_len))
            time_diff[1, carid, :] = forecasts_et[lap][carno][1].reshape((prediction_len))

        #calculate rank    
        idx = np.argsort(time_diff[0], axis=0)
        true_rank = np.argsort(idx, axis=0)

        idx = np.argsort(time_diff[1], axis=0)
        pred_rank = np.argsort(idx, axis=0)

        rank_ret.append([lap, time_diff, true_rank, pred_rank])
        
    return rank_ret,forecasts_et
    
#calc rank
def eval_rank(test_ds,tss,forecasts,prediction_length, start_offset):
    """
    evaluate rank by laptime forecasting
    
    input:
        test_ds       ; must be test set for a single event, because test_ds itself does not 
                        contain features to identify the eventid
        start_offset[]; elapsed time for lap0, for one specific event
        tss,forecasts ; forecast result
        prediction_length ; 
    return:
        rank_ret      ;  [lap, elapsed_time, true_rank, pred_rank] 
        forecasts_et  ;  {[completed_laps][carno]} ->(elapsed_time, elapsed_time_pred)
        
    """

    carlist = []

    # carno-lap# -> elapsed_time[] array
    forecasts_et = dict()

    ds_iter =  iter(test_ds)
    for idx in range(len(test_ds)):
        test_rec = next(ds_iter)
        #global carid
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        #print('car no:', carno)

        if carno not in carlist:
            carlist.append(carno)

        #start_offset is global var
        offset = start_offset[(start_offset['car_number']==carno)].elapsed_time.values[0] 
        #print('start_offset:', offset)

        # calc elapsed time
        prediction_len = forecasts[idx].samples.shape[1]
        if prediction_length != prediction_len:
            print('error: prediction_len does not match, {prediction_length}:{prediction_len}')
            return []
        
        forecast_laptime_mean = np.mean(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        #forecast_laptime_mean = np.median(forecasts[idx].samples, axis=0).reshape((prediction_len,1))

        laptime_array = tss[idx].values.copy()
        elapsed_time = np.cumsum(laptime_array) + offset

        laptime_array = tss[idx].values.copy()
        laptime_array[-prediction_len:] = forecast_laptime_mean 
        elapsed_time_hat = np.cumsum(laptime_array) + offset

        #save the prediction
        completed_laps = len(tss[idx]) - prediction_len + 1
        #print('car no:', carno, 'completed_laps:', completed_laps)
        #key = '%s-%s'%(carno, completed_laps)
        #forecasts_et[key] = elapsed_time[-prediction_len:].copy()
        if completed_laps not in forecasts_et:
            forecasts_et[completed_laps] = {}
        #forecasts_et[completed_laps][carno] = [elapsed_time[-prediction_len-1:].copy(),
        #                                           elapsed_time_hat[-prediction_len-1:].copy()]
        forecasts_et[completed_laps][carno] = [elapsed_time[-prediction_len:].copy(),
                                                   elapsed_time_hat[-prediction_len:].copy()]


    # calc rank
    rank_ret = []
    for lap in forecasts_et.keys():
        #get car list for this lap
        carlist = list(forecasts_et[lap].keys())
        #print('carlist:', carlist)

        caridmap={key:idx for idx, key in enumerate(carlist)}

        #fill in data
        #elapsed_time = np.zeros((2, len(carlist), prediction_len+1))
        elapsed_time = np.zeros((2, len(carlist), prediction_len))
        for carno in carlist:
            carid = caridmap[carno]
            elapsed_time[0, carid, :] = forecasts_et[lap][carno][0]
            elapsed_time[1, carid, :] = forecasts_et[lap][carno][1]

        #calculate rank    
        idx = np.argsort(elapsed_time[0], axis=0)
        true_rank = np.argsort(idx, axis=0)

        idx = np.argsort(elapsed_time[1], axis=0)
        pred_rank = np.argsort(idx, axis=0)

        rank_ret.append([lap, elapsed_time, true_rank, pred_rank])
        
    return rank_ret,forecasts_et
   
def get_acc(rank_ret,prediction_length):   
    """
    input:
        rank_ret: [lap, elapsed_time, true_rank, pred_rank], use [2][3] columns
    return:
        ((metrics...)
         (record count...))
         
    the result can be used to calculate micro/macro metrics
    """
    # evaluate
    #top1 accuracy
    top1acc = 0
    top1acc_farmost = 0
    top5acc = 0
    top5acc_farmost = 0
    tau = 0
    rmse = 0.
    
    for rec in rank_ret:
        trueRank = rec[2]
        predRank = rec[3]

        #top1 , rank = 0, first col is not prediction
        top1acc += np.sum((trueRank==0) & (predRank==0)) 
        
        top1acc_farmost += np.sum((trueRank[:,-1]==0) & (predRank[:,-1]==0))
        
        #top5
        top5acc += np.sum((trueRank<5) & (predRank<5)) 
        
        top5acc_farmost += np.sum((trueRank[:,-1]<5) & (predRank[:,-1]<5))
        
        # tau
        tao, _ = stats.kendalltau(trueRank, predRank)
        tau += tao
        
        #rmse
        rmse += mean_squared_error(predRank,trueRank)
        
    recnt = len(rank_ret)
    if recnt > 0:
        top1acc = top1acc *1.0/ (recnt*prediction_length)
        top1acc_farmost = top1acc_farmost *1.0/ recnt
        top5acc = top5acc *1.0/ (5*recnt*prediction_length)
        top5acc_farmost = top5acc_farmost *1.0/ (5*recnt)
        tau = tau/recnt
        rmse = rmse/recnt


        print(f'total:{len(rank_ret)}, prediction_length:{prediction_length}') 
        print('top1acc=', top1acc,
              'top1acc_farmost=', top1acc_farmost,
              'top5acc=', top5acc,
              'top5acc_farmost=', top5acc_farmost,
             )
        print('tau = ', tau,
             'rmse = ', rmse)
    
    return ((top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse),
            (recnt*prediction_length,recnt,5*recnt*prediction_length,5*recnt,recnt,recnt))
    

In [104]:
def run_exp(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = [], datamode = MODE_ORACLE,models = ['oracle']):
    """
    dependency: test_event, test on one event only
    
    """
    retdf = []
    pred_ret = {}
    ds_ret = {}
    rank_result = {}
    
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=datamode,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('exp:',inspect.stack()[0][3],'model:', model, 'datamode:', get_modestr(datamode))
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds

        rank_ret, forecast_ret = eval_rank(test_ds,tss,forecasts,prediction_length,global_start_offset[test_event])
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,get_modestr(datamode),trainid]
        ret.extend(metrics[0])
        retdf.append(ret)
        
        rank_result[model] = (rank_ret,forecast_ret)
    
    return pred_ret, ds_ret, rank_result, retdf

In [13]:
def run_exp_baseline(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):           
    """
    dependency: test_event, test on one event only
    
    """
    
    models = ['oracle','deepAR','naive']
    #,'arima']
    retdf = []
    pred_ret = {}
    ds_ret = {}
    rank_result = {}
    
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_ORACLE,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds

        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length,global_start_offset[test_event])
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics[0])
        retdf.append(ret)
        
        rank_result[model] = rank_ret
    
    # special model with test_ds
    models = ['curtrack']        
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_TESTCURTRACK,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds
        
        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length,global_start_offset[test_event])
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics[0])
        retdf.append(ret)
        
        rank_result[model] = rank_ret

    # zerotrack
    models = ['zerotrack']        
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds
        
        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length,global_start_offset[test_event])
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics[0])
        retdf.append(ret)
        
        rank_result[model] = rank_ret
    
        
    return pred_ret, ds_ret, rank_result, retdf

In [20]:
def run_exp_predtrack(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                                        test_event='Indy500', test_cars = []):
    models = ['oracle','curtrack','zerotrack']
    datamode = MODE_PREDTRACK
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)
    

def run_exp_predpit(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):
    models = ['oracle','curtrack','zerotrack']
    datamode=MODE_PREDPIT
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)

#MODE_DISTURB_CLEARTRACK = 64
#MODE_DISTURB_ADJUSTTRACK = 128
#MODE_DISTURB_ADJUSTPIT = 256
def run_exp_cleartrack(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):
    models = ['oracle']
    datamode=MODE_DISTURB_CLEARTRACK
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)

def run_exp_adjusttrack(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):
    """
    dependency: test_event, test on one event only
    
    """
    models = ['oracle']
    datamode=MODE_DISTURB_CLEARTRACK + MODE_DISTURB_ADJUSTTRACK
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)

def run_exp_adjustpit(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):
    models = ['oracle']
    datamode=MODE_DISTURB_ADJUSTPIT
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)
    
def run_exp_adjustall(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
                   test_event='Indy500', test_cars = []):
    models = ['oracle']
    datamode=MODE_DISTURB_CLEARTRACK +MODE_DISTURB_ADJUSTPIT +MODE_DISTURB_ADJUSTTRACK
    return run_exp(prediction_length, half_moving_win, train_ratio=train_ratio, trainid=trainid,
                   test_event=test_event, test_cars = test_cars, datamode = datamode,models = models)    


In [98]:
def run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode='', models=[]):
    """
    
    input:
        plens=[2,5,10]
        half=[False]
        #trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
        trainids = ["r0.5"]
        #half=[True,False]
        #plens=[2]
        runs = 5
        train_ratio=0.5 
        exp_id='mean-splitbystage-predpit'
        
        testfunc ; run_exp_predpit, run_exp_predtrack, run_exp ...

    return:
    
        dfret  ; average result of multiple runs
                 dataframe['model' , 'prediction_length', 'halfmode','trainid',
                         'top1acc','top1acc_farmost','top5acc','top5acc_farmost','tau','rmse',
                         'top1acc_std','top1acc_farmost_std','top5acc_std','top5acc_farmost_std','tau_std','rmse_std']
                          
        alldata_ret  ; for debug
            [runid][halfmode,plen,trainid] -> (pred_ret, test_ds, rank_ret)
                pred_ret[model] ->　[tss, forecasts]
                test_ds[model] ->　test_ds
                rank_ret[model] -> ([lap, elapsed_time, true_rank, pred_rank],forecast_ret)
                    forecast_ret[completed_laps][carno] -> (elapsed_time, elapsed_time_hat)
    
    """
    if plens == [] or half == [] or trainids == []:
        print("error with empty settings")
        return
    
    #testfunc or (datamode & models)
    if isinstance(testfunc,str) and (datamode == '' or models == []):
        print("error with testfunc")
        return

    allret = []
    alldata_ret = []
    for runid in range(runs):
        exp_data = []
        exp_result = []

        for halfmode in half:
            for plen in plens:
                for trainid in trainids:
                    print('='*10)
                    if not isinstance(testfunc,str):
                        pred_ret, test_ds, rank_ret, metric_ret = testfunc(plen, halfmode, 
                                                            train_ratio=train_ratio,
                                                            trainid=trainid)
                    else:
                        pred_ret, test_ds, rank_ret, metric_ret = run_exp(plen, halfmode, 
                                                            train_ratio=train_ratio,
                                                            trainid=trainid, 
                                                            datamode=datamode,
                                                            models=models)
                        

                    #save 
                    exp_data.append((pred_ret, test_ds, rank_ret))
                    exp_result.extend(metric_ret)

        #save result
        result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                           'trainid',
                                           'top1acc','top1acc_farmost','top5acc',
                                           'top5acc_farmost','tau','rmse'])

        #result['runid'] = [runid for x in range(len(result))]
        allret.append(result)
        alldata_ret.append(exp_data)

    #final
    rowcnt = len(allret[0])
    metrics = np.empty((runs, rowcnt, 6))
    for runid, ret in enumerate(allret):
        metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                           'top5acc_farmost','tau','rmse']].values


    #average
    averagemat = np.mean(metrics[:,:,:], axis=0)
    stdmat = np.std(metrics[:,:,:], axis=0)
    dfhead = allret[0][['model' , 'prediction_length', 'halfmode', 'trainid']]
    
    
    dfaverage = pd.DataFrame(averagemat, columns = ['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    dfstd = pd.DataFrame(stdmat, columns = ['top1acc_std','top1acc_farmost_std','top5acc_std',
                                       'top5acc_farmost_std','tau_std','rmse_std'])
    dfret = pd.concat([dfhead, dfaverage, dfstd], axis=1)

    if exp_id != '':
        dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-result.csv', float_format='%.3f')

    return dfret, alldata_ret


def checkret_status(dataret, runid = 0, idx = 0):
    """
    check the test_ds track and lap status
    
        alldata_ret  ; for debug
            [runid][halfmode,plen,trainid] -> (pred_ret, test_ds, rank_ret)
                pred_ret[model] ->　[tss, forecasts]
                test_ds[model] ->　test_ds
                rank_ret[model] -> ([lap, elapsed_time, true_rank, pred_rank],forecast_ret)
                    forecast_ret[completed_laps][carno] -> (elapsed_time, elapsed_time_hat)
    
    """

    _, plen = dataret[runid][idx][0]['oracle'][1][0].samples.shape
    test_ds = dataret[runid][idx][1]['oracle']
   
    
    ds_iter =  iter(test_ds)
    yfcnt = 0
    pitcnt = 0
    for recid in range(len(test_ds)):
        test_rec = next(ds_iter)
        
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        
        track_rec,lap_rec = test_rec['feat_dynamic_real']
        yfcnt += np.sum(track_rec[-plen:])
        pitcnt += np.sum(lap_rec[-plen:])
        
    print('yfcnt:', yfcnt, 'pitcnt:',pitcnt)

def get_ref_oracle_testds_single(prediction_length, half_moving_win, train_ratio=0.8, 
                   test_event='Indy500', test_cars = []):           
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_ORACLE,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)    
    return test_ds

def get_ref_oracle_testds(plens, halfs, train_ratio=0.8, 
                   test_event='Indy500', test_cars = []):           
    
    testset = {}
    for prediction_length in plens:
        for half_moving_win in halfs:
            train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_ORACLE,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)    
            
            # get key
            key = '%d-%d'%(prediction_length,half_moving_win)
            testset[key] = test_ds
            
    return testset


def checkret_confusionmat_single(dataret, ref_oracle_testds, runid= 0, idx = 0):
    """
    output the 4x4 confusion matrix split by track and lap status
    
    input:
        ref_oracle_testds  ; oracle test ds
    
    """
    _, plen = dataret[runid][idx][0]['oracle'][1][0].samples.shape
    test_ds = dataret[runid][idx][1]['oracle']
    rank_ret = dataret[runid][idx][2]['oracle']
    
    if len(ref_oracle_testds) != len(test_ds):
        print('error, size of testds mismatch', len(ref_oracle_testds), len(test_ds))
        return
    
    # confusion matrix for <trackstatus, lapstatus> type: 00,01,10,11
    # lap(start lap of prediction)  -> type
    lapmap = {}
    ds_iter =  iter(ref_oracle_testds)
    for recid in range(len(ref_oracle_testds)):
        test_rec = next(ds_iter) 
        
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        
        track_rec,lap_rec = test_rec['feat_dynamic_real']
        yfcnt = np.sum(track_rec[-plen:])
        pitcnt = np.sum(lap_rec[-plen:])    
        
        #laptype = ('0' if yfcnt==0 else '1') + ('0' if pitcnt==0 else '1')
        
        lap = len(track_rec) - plen + 1
        if lap not in lapmap:
            #lapmap[lap] = laptype
            lapmap[lap] = (yfcnt, pitcnt)
        else:
            oldtype = lapmap[lap]
            lapmap[lap] = (yfcnt + oldtype[0], pitcnt + oldtype[1])
            
    
    #split the rank_ret by laptype
    types=['00','10','01','11']
    acc_ret = []
    for laptype in types:
        check_ret = []
        for item in rank_ret:
            typecnt = lapmap[item[0]]
            
            thetype = ('0' if typecnt[0]==0 else '1') + ('0' if typecnt[1]==0 else '1')
            
            if thetype == laptype:
                check_ret.append(item)
        # get acc
        metrics = get_acc(check_ret,plen)
        recret = [laptype, len(check_ret)]
        recret.extend(metrics[0])
        acc_ret.append(recret)
    
    dfacc = pd.DataFrame(acc_ret, columns = ['type','reccnt','top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    return dfacc



def checkret_confusionmat(dataret, ref_testset, runid= 0, testid = ''):
    """
    output the 4x4 confusion matrix split by track and lap status
    
    input:
        ref_oracle_testds  ; oracle test ds
    
    """
    plen_length = len(dataret[runid])
    
    dflist = []
    for idx in range(plen_length):
        _, plen = dataret[runid][idx][0]['oracle'][1][0].samples.shape
        test_ds = dataret[runid][idx][1]['oracle']
        rank_ret = dataret[runid][idx][2]['oracle']

        key = '%d-%d'%(plen,0)
        if key not in ref_testset:
            print(f'error, {key} not found in ref_testset')
            continue
        
        ref_oracle_testds = ref_testset[key]
        if len(ref_oracle_testds) != len(test_ds):
            print('error, size of testds mismatch', len(ref_oracle_testds), len(test_ds))
            continue

        # confusion matrix for <trackstatus, lapstatus> type: 00,01,10,11
        # lap(start lap of prediction)  -> type
        lapmap = {}
        ds_iter =  iter(ref_oracle_testds)
        for recid in range(len(ref_oracle_testds)):
            test_rec = next(ds_iter) 

            carno = decode_carids[test_rec['feat_static_cat'][0]]

            track_rec,lap_rec = test_rec['feat_dynamic_real']
            yfcnt = np.sum(track_rec[-plen:])
            pitcnt = np.sum(lap_rec[-plen:])    

            #laptype = ('0' if yfcnt==0 else '1') + ('0' if pitcnt==0 else '1')

            lap = len(track_rec) - plen + 1
            if lap not in lapmap:
                #lapmap[lap] = laptype
                lapmap[lap] = (yfcnt, pitcnt)
            else:
                oldtype = lapmap[lap]
                lapmap[lap] = (yfcnt + oldtype[0], pitcnt + oldtype[1])


        #split the rank_ret by laptype
        types=['00','10','01','11']
        acc_ret = []
        for laptype in types:
            check_ret = []
            for item in rank_ret:
                typecnt = lapmap[item[0]]

                thetype = ('0' if typecnt[0]==0 else '1') + ('0' if typecnt[1]==0 else '1')

                if thetype == laptype:
                    check_ret.append(item)
            # get acc
            metrics = get_acc(check_ret,plen)
            recret = [testid, plen, laptype, len(check_ret)]
            recret.extend(metrics[0])
            acc_ret.append(recret)

        _dfacc = pd.DataFrame(acc_ret, columns = ['testid','plen',
                                'type','reccnt','top1acc','top1acc_farmost','top5acc',
                                'top5acc_farmost','tau','rmse'])
        
        dflist.append(_dfacc)
    
    dfacc = pd.concat(dflist, axis=0)
    
    return dfacc

### init

In [21]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events = ['Gateway']

#events = ['Indy500']
#events = ['Phoenix']
events_id={key:idx for idx, key in enumerate(events)}
#works for only one event


In [22]:
stagedata = {}
global_start_offset = {}
global_carids = {}
traindata = None

cur_carid = 0
for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event, year)
    
    alldata, rankdata, acldata = stagedata[event]
    #carlist = set(acldata['car_number'])
    #laplist = set(acldata['completed_laps'])
    #print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #offset
    global_start_offset[event] = rankdata[rankdata['completed_laps']==0][['car_number','elapsed_time']]
    
    #build the carid map
    #for car in carlist:
    #    if car not in global_carids:
    #        global_carids[car] = cur_carid
    #        cur_carid += 1

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
# start from here
import pickle
with open('laptime_rank_timediff_pit-oracle-%s.pickle'%year, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1')

In [24]:
freq = "1min"
#decode global_carids
decode_carids={carid:carno for carno, carid in global_carids.items()}
    
#useeid = False
#interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
#ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
#if useeid:
#    cardinality = [len(global_carids), len(laptime_data)]
#else:
#    cardinality = [len(global_carids)]

In [62]:
### test
#half=[True, False]
#plens=[2,5,10,20,30]
plens=[2]
half=[0]
trainids = ["indy500"]
#trainids = ["r0.5","r0.6"]
runs = 1
train_ratio=0.4

#trainids = ["indy500"]
#runs = 1
#plens=[2]

testfunc = 'notracklap'
datamode = MODE_NOTRACK + MODE_NOLAP
datamode = MODE_ORACLE
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)

#check the car list number for -100 test record
# it should be 24
print('carlist shape in testset:',ret[0][0][2]['oracle'][-100][2].shape)

checkret_status(ret)

sys.exit(0)

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

NameError: name 'sys' is not defined

In [ ]:
#    prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8",
#                   test_event='Indy500', test_cars = []):    
ref_testds = get_ref_oracle_testds(plens[0], half[0], trainid=trainids[0], train_ratio=train_ratio)

dfacc = checkret_confusionmat(ret, ref_testds)


In [75]:
dfacc = checkret_confusionmat(ret, ref_testds)

total:27, prediction_length:2
top1acc= 0.8333333333333334 top1acc_farmost= 0.8148148148148148 top5acc= 0.9111111111111111 top5acc_farmost= 0.9037037037037037
tau =  0.9040370813400228 rmse =  4.998719203008726
total:11, prediction_length:2
top1acc= 0.5 top1acc_farmost= 0.45454545454545453 top5acc= 0.7545454545454545 top5acc_farmost= 0.6727272727272727
tau =  0.7355163395380785 rmse =  19.380142450142447
total:49, prediction_length:2
top1acc= 0.9183673469387755 top1acc_farmost= 0.9183673469387755 top5acc= 0.9224489795918367 top5acc_farmost= 0.9102040816326531
tau =  0.9186083476358234 rmse =  2.6839480551269546
total:31, prediction_length:2
top1acc= 0.532258064516129 top1acc_farmost= 0.3870967741935484 top5acc= 0.7967741935483871 top5acc_farmost= 0.7419354838709677
tau =  0.821212595777813 rmse =  11.328675738837028


In [76]:
dfacc

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.833333,0.814815,0.911111,0.903704,0.904037,4.998719
1,10,11,0.500000,0.454545,0.754545,0.672727,0.735516,19.380142
2,01,49,0.918367,0.918367,0.922449,0.910204,0.918608,2.683948
3,11,31,0.532258,0.387097,0.796774,0.741935,0.821213,11.328676


In [77]:
testfunc = 'notracklap'
datamode = MODE_NOTRACK + MODE_NOLAP
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret2 = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
dfacc2 = checkret_confusionmat(ret2, ref_testds)
dfacc2

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.851852,0.814815,0.922222,0.911111,0.909469,4.647735
1,10,11,0.772727,0.818182,0.790909,0.781818,0.774356,17.484345
2,01,49,0.571429,0.489796,0.789796,0.730612,0.807605,14.294862
3,11,31,0.661290,0.580645,0.858065,0.800000,0.869828,7.734145


In [85]:
testfunc = 'laponly'
datamode = MODE_ORACLE_LAPONLY
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret3 = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
dfacc3 = checkret_confusionmat(ret3, ref_testds)
dfacc3

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.814815,0.777778,0.918519,0.896296,0.894066,6.249313
1,10,11,0.727273,0.727273,0.809091,0.800000,0.752895,19.401638
2,01,49,0.918367,0.897959,0.924490,0.910204,0.922125,2.388085
3,11,31,0.806452,0.741935,0.903226,0.870968,0.898505,4.219467


In [80]:
dfacc

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.833333,0.814815,0.911111,0.903704,0.904037,4.998719
1,10,11,0.500000,0.454545,0.754545,0.672727,0.735516,19.380142
2,01,49,0.918367,0.918367,0.922449,0.910204,0.918608,2.683948
3,11,31,0.532258,0.387097,0.796774,0.741935,0.821213,11.328676


In [81]:
dfacc2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.851852,0.814815,0.922222,0.911111,0.909469,4.647735
1,10,11,0.772727,0.818182,0.790909,0.781818,0.774356,17.484345
2,01,49,0.571429,0.489796,0.789796,0.730612,0.807605,14.294862
3,11,31,0.661290,0.580645,0.858065,0.800000,0.869828,7.734145


In [89]:
testfunc = 'predpit'
datamode = MODE_PREDPIT + MODE_ORACLE_LAPONLY
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret4 = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
dfacc4 = checkret_confusionmat(ret4, ref_testds)
dfacc4

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.759259,0.703704,0.866667,0.844444,0.834260,13.917916
1,10,11,0.727273,0.636364,0.809091,0.800000,0.752897,19.621742
2,01,49,0.591837,0.489796,0.804082,0.742857,0.820549,13.371057
3,11,31,0.693548,0.645161,0.864516,0.812903,0.853684,8.701912


In [91]:
testfunc = 'predpit'
datamode = MODE_PREDTRACK + MODE_ORACLE_TRACKONLY
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret5 = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
dfacc5 = checkret_confusionmat(ret5, ref_testds)
dfacc5

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.833333,0.851852,0.918519,0.903704,0.906397,4.500532
1,10,11,0.500000,0.454545,0.772727,0.763636,0.780737,14.491928
2,01,49,0.571429,0.489796,0.783673,0.730612,0.806295,14.462357
3,11,31,0.596774,0.516129,0.770968,0.696774,0.822537,10.856118


In [94]:
testfunc = 'predpit'
datamode = MODE_PREDTRACK + MODE_PREDPIT
models = ['oracle']
exp_id=f'mean-splitbyevent-test-r{runs}-t{train_ratio}'
df, ret6 = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
dfacc6 = checkret_confusionmat(ret6, ref_testds)
dfacc6

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,00,27,0.759259,0.740741,0.818519,0.792593,0.810725,17.557617
1,10,11,0.590909,0.545455,0.763636,0.745455,0.774834,14.748642
2,01,49,0.591837,0.489796,0.783673,0.726531,0.814720,13.957174
3,11,31,0.467742,0.354839,0.758065,0.696774,0.780934,15.599299


In [90]:
checkret_status(ret4)

yfcnt: 0.0 pitcnt: 46.0


### oracle test

In [99]:
### test
#half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[0]
trainids = ["indy500"]
#trainids = ["r0.5","r0.6"]
runs = 1
train_ratio=0.4

#trainids = ["indy500"]
#runs = 1
#plens=[2]
exp_id=''

config = {'fulloracle':MODE_ORACLE,'notracklap':MODE_NOTRACK + MODE_NOLAP,
         'laponly':MODE_ORACLE_LAPONLY, 'trackonly':MODE_ORACLE_TRACKONLY,
          'fullpred':MODE_PREDTRACK + MODE_PREDPIT,
          'predtrack':MODE_PREDTRACK + MODE_ORACLE_TRACKONLY,
          'predpit':MODE_PREDPIT + MODE_ORACLE_LAPONLY          
         }

ref_testset = get_ref_oracle_testds(plens, half, train_ratio=train_ratio)

acclist = []
dflist = []
for teststr in config.keys():
    testfunc = teststr
    datamode = config[teststr]

    models = ['oracle']
    df, dataret = run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)
    
    #concat
    acc = checkret_confusionmat(dataret, ref_testset, testid = teststr)
    dflist.append(df)
    acclist.append(acc)
    
dfret = pd.concat(dflist, axis=0)
dfacc = pd.concat(acclist, axis=0)

train len:0, test len:3142
train len:0, test len:3055
train len:0, test len:2910
train len:0, test len:3142


INFO:root:Using GPU


exp: run_exp model: oracle
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
tss len=3142, forecasts len=3142


INFO:root:Using GPU


total:118, prediction_length:2
top1acc= 0.75 top1acc_farmost= 0.7203389830508474 top5acc= 0.8601694915254238 top5acc_farmost= 0.823728813559322
tau =  0.8728054182929791 rmse =  6.960997717703307
train len:0, test len:3055
exp: run_exp model: oracle
predicting model=oracle, plen=5
loading model...done!, ctx:gpu(0)
tss len=3055, forecasts len=3055


...

train len:0, test len:3142
exp: run_exp model: oracle
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
tss len=3142, forecasts len=3142


INFO:root:Using GPU


total:118, prediction_length:2
top1acc= 0.6440677966101694 top1acc_farmost= 0.5677966101694916 top5acc= 0.8228813559322034 top5acc_farmost= 0.7711864406779662
tau =  0.8198457831555875 rmse =  13.699878109154273
train len:0, test len:3055
exp: run_exp model: oracle
predicting model=oracle, plen=5
loading model...done!, ctx:gpu(0)
tss len=3055, forecasts len=3055


INFO:root:Using GPU


total:115, prediction_length:5
top1acc= 0.47304347826086957 top1acc_farmost= 0.3130434782608696 top5acc= 0.672695652173913 top5acc_farmost= 0.5026086956521739
tau =  0.6648644200213047 rmse =  29.744838395984903
train len:0, test len:2910
exp: run_exp model: oracle
predicting model=oracle, plen=10
loading model...done!, ctx:gpu(0)
tss len=2910, forecasts len=2910
total:110, prediction_length:10
top1acc= 0.35454545454545455 top1acc_farmost= 0.15454545454545454 top5acc= 0.5492727272727272 top5acc_farmost= 0.3236363636363636
tau =  0.5324884199296123 rmse =  43.71305278888471
total:27, prediction_length:2
top1acc= 0.7222222222222222 top1acc_farmost= 0.6666666666666666 top5acc= 0.8407407407407408 top5acc_farmost= 0.8
tau =  0.8152245822814133 rmse =  16.798760575620477
total:11, prediction_length:2
top1acc= 0.6818181818181818 top1acc_farmost= 0.6363636363636364 top5acc= 0.7818181818181819 top5acc_farmost= 0.7818181818181819
tau =  0.752712746191007 rmse =  19.547420357420357
total:49, pred

### loop test

In [ ]:
test_result = {}

In [ ]:
plens=[2,5,10]
half=[0]
trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
#trainids = ["r0.5","r0.6"]
runs = 5
train_ratio=0.4

testfunc = 'trackonly'
datamode = MODE_ORACLE_TRACKONLY
models = ['oracle']
exp_id=f'mean-splitbyevent-{testfunc}-r{runs}-t{train_ratio}'
run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)

testfunc = 'laponly'
datamode = MODE_ORACLE_LAPONLY
models = ['oracle']
exp_id=f'mean-splitbyevent-{testfunc}-r{runs}-t{train_ratio}'
run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)


In [44]:
plens=[2,5,10]
half=[0]
trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
#trainids = ["r0.5","r0.6"]
runs = 5
train_ratio=0.4

testfunc = 'notracklap'
datamode = MODE_NOTRACK + MODE_NOLAP
models = ['oracle']
exp_id=f'mean-splitbyevent-{testfunc}-r{runs}-t{train_ratio}'
run_test(runs, plens, half, trainids, train_ratio, testfunc, datamode=datamode,models=models)


INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carno:21, totallen:199, nancount:1, test_reccnt:117
carno:22, totallen:200, nancount:0, test_reccnt:118
carno:23, totallen:200, nancount:0, test_reccnt:118
carno:24, totallen:154, nancount:46, test_reccnt:72
carno:2

INFO:root:Using GPU


total:118, prediction_length:2
top1acc= 0.6991525423728814 top1acc_farmost= 0.6271186440677966 top5acc= 0.826271186440678 top5acc_farmost= 0.7864406779661017
tau =  0.8423388484583537 rmse =  10.924750311667703
====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:118
carno:3, totallen:146, nancount:54, test_reccnt:64
carno:4, totallen:200, nancount:0, test_reccnt:118
carno:6, totallen:200, nancount:0, test_reccnt:118
carno:7, totallen:193, nancount:7, test_reccnt:111
carno:9, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:118
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:105
carno:15, totallen:200, nancount:0, test_reccnt:118
carno:17, totallen:199, nancount:1, test_reccnt:117
carno:18, totallen:137, nancount:63, test_reccnt:55
carno:19, totallen:199, nancount:1, test_reccnt:117
carno:20, totallen:200, nancount:0, test_reccnt:118
carn

INFO:root:Using GPU


...

total:115, prediction_length:5
top1acc= 0.5443478260869565 top1acc_farmost= 0.4 top5acc= 0.7318260869565217 top5acc_farmost= 0.5930434782608696
tau =  0.7300298473730392 rmse =  21.845696567451256
====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:110
carno:3, totallen:146, nancount:54, test_reccnt:56
carno:4, totallen:200, nancount:0, test_reccnt:110
carno:6, totallen:200, nancount:0, test_reccnt:110
carno:7, totallen:193, nancount:7, test_reccnt:103
carno:9, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:97
carno:15, totallen:200, nancount:0, test_reccnt:110
carno:17, totallen:199, nancount:1, test_reccnt:109
carno:18, totallen:137, nancount:63, test_reccnt:47
carno:19, totallen:199, nancount:1, test_reccnt:109
carno:20, totallen:200, nancount:0, test_reccnt:110
carno:21, totallen:

INFO:root:Using GPU


total:110, prediction_length:10
top1acc= 0.35363636363636364 top1acc_farmost= 0.15454545454545454 top5acc= 0.5794545454545454 top5acc_farmost= 0.3327272727272727
tau =  0.5684317659654007 rmse =  38.40169044024995
====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:110
carno:3, totallen:146, nancount:54, test_reccnt:56
carno:4, totallen:200, nancount:0, test_reccnt:110
carno:6, totallen:200, nancount:0, test_reccnt:110
carno:7, totallen:193, nancount:7, test_reccnt:103
carno:9, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:97
carno:15, totallen:200, nancount:0, test_reccnt:110
carno:17, totallen:199, nancount:1, test_reccnt:109
carno:18, totallen:137, nancount:63, test_reccnt:47
carno:19, totallen:199, nancount:1, test_reccnt:109
carno:20, totallen:200, nancount:0, test_reccnt:110
ca

INFO:root:Using GPU


total:110, prediction_length:10
top1acc= 0.35363636363636364 top1acc_farmost= 0.13636363636363635 top5acc= 0.5581818181818182 top5acc_farmost= 0.3
tau =  0.5450098887292851 rmse =  40.34333463256716
====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:110
carno:3, totallen:146, nancount:54, test_reccnt:56
carno:4, totallen:200, nancount:0, test_reccnt:110
carno:6, totallen:200, nancount:0, test_reccnt:110
carno:7, totallen:193, nancount:7, test_reccnt:103
carno:9, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:110
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:97
carno:15, totallen:200, nancount:0, test_reccnt:110
carno:17, totallen:199, nancount:1, test_reccnt:109
carno:18, totallen:137, nancount:63, test_reccnt:47
carno:19, totallen:199, nancount:1, test_reccnt:109
carno:20, totallen:200, nancount:0, test_reccnt:110
carno:21, totalle

In [ ]:
#half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[0]
trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
#trainids = ["r0.5","r0.6"]
runs = 5
train_ratio=0.4

#trainids = ["indy500"]
#runs = 1
#plens=[2]

testfunc = run_exp_cleartrack
exp_id=f'mean-splitbyevent-adjustcleartrack-r{runs}-t{train_ratio}'
test_result[exp_id] = run_test(runs, plens, half, trainids, train_ratio, testfunc)

testfunc = run_exp_adjusttrack
exp_id=f'mean-splitbyevent-adjusttrack-r{runs}-t{train_ratio}'
test_result[exp_id] = run_test(runs, plens, half, trainids, train_ratio, testfunc)

testfunc = run_exp_adjustpit
exp_id=f'mean-splitbyevent-adjustpit-r{runs}-t{train_ratio}'
test_result[exp_id] = run_test(runs, plens, half, trainids, train_ratio, testfunc)

testfunc = run_exp_adjustall
exp_id=f'mean-splitbyevent-adjustall-r{runs}-t{train_ratio}'
test_result[exp_id] = run_test(runs, plens, half, trainids, train_ratio, testfunc)

#testfunc = run_exp
#exp_id=f'mean-splitbyevent-baseline-r{runs}-t{train_ratio}'
#test_result[exp_id] = run_test(runs, plens, half, trainids, train_ratio, testfunc)


In [ ]:
#half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[0]
#trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
trainids = ["r0.5","r0.6","r0.7"]
runs = 5
train_ratio=0.7



### test

In [42]:
#check the car list number for -100 test record
ret[0][0][2]['oracle'][-100][2].shape


(24, 10)

In [37]:
type(ret[0][0][2])

dict

In [73]:
a=(3,4)
a+(1,2)

(3, 4, 1, 2)

In [103]:
dfacc[dfacc['plen']==2]

,testid,plen,type,reccnt,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,fulloracle,2,00,27,0.814815,0.814815,0.922222,0.903704,0.911145,4.437743
1,fulloracle,2,10,11,0.409091,0.363636,0.690909,0.636364,0.741955,18.065659
2,fulloracle,2,01,49,0.908163,0.918367,0.920408,0.906122,0.919519,2.650620
3,fulloracle,2,11,31,0.564516,0.451613,0.770968,0.690323,0.812005,12.031485
0,notracklap,2,00,27,0.851852,0.814815,0.925926,0.911111,0.910041,4.603267
1,notracklap,2,10,11,0.772727,0.818182,0.781818,0.781818,0.774887,17.558186
2,notracklap,2,01,49,0.581633,0.510204,0.791837,0.738776,0.808444,14.227766
3,notracklap,2,11,31,0.661290,0.645161,0.861290,0.812903,0.870466,7.505342
0,laponly,2,00,27,0.851852,0.814815,0.918519,0.896296,0.897437,6.049208
1,laponly,2,10,11,0.772727,0.727273,0.790909,0.800000,0.755912,19.354696
